# 🧹 Tratamento e Transformação das Tabelas

## 🎯 Objetivo do Notebook

Este notebook executa as seguintes tarefas principais:

---

### 1. 🧱 Tratamento da Tabela Base

Realiza o tratamento e a definição da **tabela base**, com base no dataset `train_base`.

---

### 2. 🛰️ Tratamento das Tabelas Satélites Menos Relevantes

Executa tratamento **básico e padronizado** para datasets satélites com menor impacto direto na modelagem.

---

### 3. 🌐 Tratamento das Tabelas Satélites Mais Relevantes

Aplica transformações e enriquecimentos em **datasets satélites considerados mais relevantes** para construir uma tabela base mais informativa e robusta.

---


### 4. 📖 Atualização de Dicionário de Variáveis (em Português)

Gera um **mapeamento interpretável** com descrições das variáveis criadas nesta fase, tornando a análise mais acessível e compreensível.


In [2]:
# Criação da SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploSparkSession") \
    .getOrCreate()

In [48]:
# Importações
from pyspark.sql.functions import col, when, count, isnan, isnull, countDistinct, to_date, format_string, to_date, datediff, lit, coalesce
from pyspark.sql.types import FloatType, DoubleType
import pandas as pd
import os
import glob

In [16]:
caminho = r"C:\Users\fred\meu_projeto_etl"

### 🧪 Função: `avaliar_dataset`

Esta função tem como objetivo realizar uma **análise exploratória detalhada** de um DataFrame Spark, focando em qualidade e integridade dos dados. É uma ferramenta útil para engenheiros e cientistas de dados durante as etapas iniciais do pipeline de dados.

#### ✅ O que a função faz:

- Exibe o **schema** (colunas e seus tipos de dados)
- Conta:
  - Total de registros
  - **Nulos e % de nulos por coluna**
  - **Valores distintos** por coluna
- Mostra **exemplos de valores** não nulos por coluna
- Verifica **unicidade da chave primária** (se especificada)
- Mostra **amostras de colunas de data**, ignorando nulos
- Retorna:
  - Um **DataFrame Pandas** com o diagnóstico completo por coluna
  - Um **dicionário resumo** com métricas globais

#### 📥 Parâmetros:
- `df`: DataFrame Spark a ser avaliado
- `pk` *(str, opcional)*: Nome da coluna chave primária a ser validada
- `colunas_data` *(list[str], opcional)*: Lista de colunas com datas para exibir amostras
- `amostras_data` *(int, opcional)*: Número de registros a exibir por coluna de data (padrão: 5)

#### 🧠 Observações:
- A verificação de duplicatas é feita **somente na chave primária** (não em registros completos, para evitar sobrecarga de memória).


In [4]:
from pyspark.sql.functions import col, isnull, isnan
import pandas as pd

def avaliar_dataset(df, pk=None, colunas_data=None, amostras_data=5):

    print("📊 Avaliação do DataFrame...")
    total_registros = df.count()
    schema = dict(df.dtypes)
    resultados = []

    for c in df.columns:
        tipo = schema[c]
        cond = isnull(col(c))
        if tipo in ("float", "double"):
            cond = cond | isnan(col(c))

        nulos = df.filter(cond).count()
        distintos = df.select(c).distinct().count()

        # Captura de exemplos distintos e não nulos
        try:
            exemplos_df = df.select(c).where(col(c).isNotNull()).distinct().limit(3)
            exemplo = exemplos_df.toPandas()[c].tolist()
        except:
            exemplo = []

        resultados.append({
            "coluna": c,
            "tipo": tipo,
            "% nulos": round(nulos / total_registros * 100, 2),
            "qtd_nulos": nulos,
            "valores_distintos": distintos,
            "exemplo_valores": exemplo,
            "observacoes": ""
        })

    resultados_df = pd.DataFrame(resultados)

    # 🔍 Resumo geral
    resumo = {
        "total_registros": total_registros,
        "colunas_com_nulos": resultados_df[resultados_df["qtd_nulos"] > 0].shape[0],
        "colunas_totalmente_nulas": resultados_df[resultados_df["qtd_nulos"] == total_registros].shape[0],
        "duplicatas_pk": None
    }

    # Verificação de chave primária
    if pk and pk in df.columns:
        duplicatas_pk = df.groupBy(pk).count().filter(col("count") > 1).count()
        resumo["duplicatas_pk"] = duplicatas_pk
        print(f"\n🔍 Chave primária '{pk}':")
        if duplicatas_pk > 0:
            print(f"⚠️ Encontradas {duplicatas_pk} duplicatas com base na chave '{pk}'")
        else:
            print("✅ Chave primária é única.")
    elif pk:
        print(f"⚠️ A coluna '{pk}' não está presente no DataFrame.")

    # Amostra de colunas de data
    if colunas_data:
        print("\n📅 Amostras de colunas de data (sem nulos):")
        for data_col in colunas_data:
            if data_col in df.columns:
                print(f"📆 {data_col}:")
                try:
                    df.filter(col(data_col).isNotNull()).select(data_col).show(amostras_data, truncate=False)
                except Exception as e:
                    print(f"⚠️ Erro ao exibir {data_col}: {e}")
            else:
                print(f"⚠️ Coluna {data_col} não encontrada.")

    return resultados_df, resumo


## 1. 🧱 Tratamento da Tabela Base

In [5]:
#Importação da tabela base
train_base = spark.read.parquet(r"C:\Users\fred\Documents\Estudo de dados\projeto_credito\data\raw\train\train_base.parquet")

### Tratamento de nulos e duplicidade

In [8]:
# Executar
tabela_resultado, resumo = avaliar_dataset(
    df=train_base,
    pk="case_id",
    colunas_data=["date_decision", "MONTH"])

# Visualizar a tabela
import pandas as pd
from IPython.display import display
display(tabela_resultado)

# Exibir resumo
print(resumo)
avaliar_dataset(train_base, pk="case_id", colunas_data=["date_decision", "MONTH"])

📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
✅ Chave primária é única.

📅 Amostras de colunas de data (sem nulos):
📆 date_decision:
+-------------+
|date_decision|
+-------------+
|2019-01-03   |
|2019-01-03   |
|2019-01-04   |
|2019-01-03   |
|2019-01-04   |
+-------------+
only showing top 5 rows

📆 MONTH:
+------+
|MONTH |
+------+
|201901|
|201901|
|201901|
|201901|
|201901|
+------+
only showing top 5 rows



,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.0,0,1526659,"[26, 29, 474]",
1,date_decision,string,0.0,0,644,"[2019-08-08, 2019-08-22, 2019-08-23]",
2,MONTH,bigint,0.0,0,22,"[201910, 201905, 202003]",
3,WEEK_NUM,bigint,0.0,0,92,"[26, 29, 65]",
4,target,bigint,0.0,0,2,"[0, 1]",


{'total_registros': 1526659, 'colunas_com_nulos': 0, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 0}
📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
✅ Chave primária é única.

📅 Amostras de colunas de data (sem nulos):
📆 date_decision:
+-------------+
|date_decision|
+-------------+
|2019-01-03   |
|2019-01-03   |
|2019-01-04   |
|2019-01-03   |
|2019-01-04   |
+-------------+
only showing top 5 rows

📆 MONTH:
+------+
|MONTH |
+------+
|201901|
|201901|
|201901|
|201901|
|201901|
+------+
only showing top 5 rows



(          coluna    tipo  % nulos  qtd_nulos  valores_distintos  \
 0        case_id  bigint      0.0          0            1526659   
 1  date_decision  string      0.0          0                644   
 2          MONTH  bigint      0.0          0                 22   
 3       WEEK_NUM  bigint      0.0          0                 92   
 4         target  bigint      0.0          0                  2   
 
                         exemplo_valores observacoes  
 0                         [26, 29, 474]              
 1  [2019-08-08, 2019-08-22, 2019-08-23]              
 2              [201910, 201905, 202003]              
 3                          [26, 29, 65]              
 4                                [0, 1]              ,
 {'total_registros': 1526659,
  'colunas_com_nulos': 0,
  'colunas_totalmente_nulas': 0,
  'duplicatas_pk': 0})

### Tratamento de tipos de dados

In [9]:
# Alteração de tipo da coluna date_decision
train_base_date = train_base.withColumn('date_decision', to_date(col('date_decision'), 'yyyy-MM-dd'))

In [10]:
# Verificação da alteração do tipo
train_base_date.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- date_decision: date (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- WEEK_NUM: long (nullable = true)
 |-- target: long (nullable = true)



In [11]:
# Verificação do formato do mês
train_base_date.select('MONTH').show()

+------+
| MONTH|
+------+
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
|201901|
+------+
only showing top 20 rows



In [12]:
# Alteração de tipo da coluna MONTH
train_base_date = train_base_date.withColumn(
    "MONTH",
    to_date(format_string("%06d", col("MONTH")), "yyyyMM")
)

In [13]:
train_base_date.select('MONTH').distinct().orderBy('MONTH').show(20, truncate=False)

+----------+
|MONTH     |
+----------+
|2019-01-01|
|2019-02-01|
|2019-03-01|
|2019-04-01|
|2019-05-01|
|2019-06-01|
|2019-07-01|
|2019-08-01|
|2019-09-01|
|2019-10-01|
|2019-11-01|
|2019-12-01|
|2020-01-01|
|2020-02-01|
|2020-03-01|
|2020-04-01|
|2020-05-01|
|2020-06-01|
|2020-07-01|
|2020-08-01|
+----------+
only showing top 20 rows



In [14]:
# Verificação da alteração do tipo
train_base_date.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- date_decision: date (nullable = true)
 |-- MONTH: date (nullable = true)
 |-- WEEK_NUM: long (nullable = true)
 |-- target: long (nullable = true)



In [17]:
# Monta o caminho completo até o diretório de saída
caminho_saida = os.path.join(caminho, "data", "interim", "train_base_tratada")

# Salva o DataFrame Spark em formato Parquet no local desejado
train_base_date.write.mode("overwrite").parquet(caminho_saida)


## 2. 🛰️  Tratamento das tabelas "Satélite"

### Tabela de descrição de variáveis

In [18]:
import pandas as pd

# Ler o arquivo markdown como texto
df_pd = pd.read_table(r"C:\Users\fred\Documents\Estudo de dados\projeto_credito\docs\tabela_descritiva.md", sep="|", engine="python", skipinitialspace=True)
df_pd = df_pd.loc[:, ~df_pd.columns.str.contains('^Unnamed')]
df_pd.columns = df_pd.columns.str.strip()
df_pd = df_pd.applymap(lambda x: x.strip() if isinstance(x, str) else x)

tabela_descritiva = spark.createDataFrame(df_pd)
tabela_descritiva.printSchema()
tabela_descritiva.show()

root
 |-- tabela: string (nullable = true)
 |-- coluna: string (nullable = true)
 |-- tipo_dado: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Descricao: string (nullable = true)

+--------------------+--------------------+-------------+--------------------+--------------------+
|              tabela|              coluna|    tipo_dado|         Description|           Descricao|
+--------------------+--------------------+-------------+--------------------+--------------------+
|:----------------...|:----------------...|:------------|:----------------...|:----------------...|
|          train_base|             case_id|       bigint|                 NaN|                 NaN|
|          train_base|       date_decision|       string|                 NaN|                 NaN|
|          train_base|               MONTH|       bigint|                 NaN|                 NaN|
|          train_base|            WEEK_NUM|       bigint|                 NaN|               

## Tratamento da tabela train_applprev_1_0

In [19]:
tabela_descritiva.columns

['tabela', 'coluna', 'tipo_dado', 'Description', 'Descricao']

In [20]:
# Filtrar e selecionar
tabela_descritiva_filtrada = tabela_descritiva.filter(
    col('tabela') == 'train_applprev_1_0'
).select('coluna', 'tipo_dado', 'Descricao')

# Mostrar para inspeção
tabela_descritiva_filtrada.show(45, truncate=False)

# Transformar em Pandas
tabela_descritiva_pd = tabela_descritiva_filtrada.toPandas()

# Salvar como Markdown
md_path = r"C:\Users\fred\Documents\Estudo de dados\projeto_credito\docs\tabela_descritiva1.md"

with open(md_path, "w", encoding="utf-8") as f:
    f.write(tabela_descritiva_pd.to_markdown(index=False))


+---------------------------+---------+--------------------------------------------------------------------------------------------------------+
|coluna                     |tipo_dado|Descricao                                                                                               |
+---------------------------+---------+--------------------------------------------------------------------------------------------------------+
|case_id                    |bigint   |NaN                                                                                                     |
|actualdpd_943P             |double   |Dias vencidos (DPD) do contrato anterior (real).                                                        |
|annuity_853A               |double   |Anuidade mensal para aplicativos anteriores.                                                            |
|approvaldate_319D          |string   |Data de aprovação do pedido anterior                                                       

In [21]:
# Carregando e unindo tabelas
train_applprev_1_0 = spark.read.parquet(r"C:\Users\fred\Documents\Estudo de dados\projeto_credito\data\raw\train\train_applprev_1_0.parquet")
train_applprev_1_1 = spark.read.parquet(r"C:\Users\fred\Documents\Estudo de dados\projeto_credito\data\raw\train\train_applprev_1_1.parquet")
train_applprev_1 = train_applprev_1_0.unionByName(train_applprev_1_1)

In [22]:
# Executar
tabela_resultado, resumo = avaliar_dataset(
    df=train_applprev_1,
    pk="case_id",
    colunas_data=["approvaldate_319D", "creationdate_885D", "dateactivated_425D", "dtlastpmt_581D","dtlastpmtallstes_3545839D", "employedfrom_700D", "firstnonzeroinstldate_307D"])

# Visualizar a tabela
import pandas as pd
from IPython.display import display
display(tabela_resultado)

# Exibir resumo
print(resumo)


📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 991955 duplicatas com base na chave 'case_id'

📅 Amostras de colunas de data (sem nulos):
📆 approvaldate_319D:
+-----------------+
|approvaldate_319D|
+-----------------+
|2019-01-11       |
|2018-10-11       |
|2018-12-31       |
|2018-11-02       |
|2018-12-11       |
+-----------------+
only showing top 5 rows

📆 creationdate_885D:
+-----------------+
|creationdate_885D|
+-----------------+
|2013-04-03       |
|2013-04-03       |
|2019-01-07       |
|2019-01-08       |
|2019-01-16       |
+-----------------+
only showing top 5 rows

📆 dateactivated_425D:
+------------------+
|dateactivated_425D|
+------------------+
|2018-10-19        |
|2018-11-07        |
|2018-12-28        |
|2018-10-09        |
|2018-11-16        |
+------------------+
only showing top 5 rows

📆 dtlastpmt_581D:
+--------------+
|dtlastpmt_581D|
+--------------+
|2019-01-10    |
|2019-01-03    |
|2019-01-08    |
|2018-12-05    |
|2018-12-26  

,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,1221522,"[29, 964, 2453]",
1,actualdpd_943P,double,0.04,2500,190,"[160.0, 8.0, 70.0]",
2,annuity_853A,double,3.84,250736,89298,"[7171.0, 3069.6, 5482.2]",
3,approvaldate_319D,string,46.13,3010294,5403,"[2019-08-08, 2019-08-22, 2017-05-14]",
4,byoccupationinc_3656910L,double,76.49,4991531,29700,"[24923.0, 45953.0, 7542.62]",
5,cancelreason_3545846M,string,0.00,0,76,"[P205_40_167, P150_0_30, P60_137_164]",
6,childnum_21L,double,54.54,3559424,21,"[8.0, 0.0, 7.0]",
7,creationdate_885D,string,0.00,66,5406,"[2016-08-17, 2015-05-01, 2017-12-05]",
8,credacc_actualbalance_314A,double,95.10,6205945,93622,"[49.598003, 157.66801, 24594.0]",
9,credacc_credlmt_575A,double,2.97,194132,52332,"[14452.0, 24594.0, 16754.6]",


{'total_registros': 6525979, 'colunas_com_nulos': 32, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 991955}


In [23]:
# Converter todas as colunas de data para DateType (assumindo formato yyyy-MM-dd)
train_applprev_1_date = train_applprev_1.withColumn("approvaldate_319D", to_date("approvaldate_319D")) \
       .withColumn("creationdate_885D", to_date("creationdate_885D")) \
       .withColumn("dateactivated_425D", to_date("dateactivated_425D")) \
       .withColumn("dtlastpmt_581D", to_date("dtlastpmt_581D")) \
       .withColumn("dtlastpmtallstes_3545839D", to_date("dtlastpmtallstes_3545839D")) \
       .withColumn("employedfrom_700D", to_date("employedfrom_700D")) \
       .withColumn("firstnonzeroinstldate_307D", to_date("firstnonzeroinstldate_307D"))


In [24]:
# Dias contados
train_applprev_1 = train_applprev_1_date.withColumn("dias_para_aprovacao", datediff("approvaldate_319D", "creationdate_885D")) \
       .withColumn("dias_ate_ativacao", datediff("dateactivated_425D", "creationdate_885D")) \
       .withColumn("dias_ult_pagamento", datediff("dtlastpmt_581D", "creationdate_885D")) \
       .withColumn("dias_ult_pagamento_all", datediff("dtlastpmtallstes_3545839D", "creationdate_885D")) \
       .withColumn("dias_desde_inicio_emprego", datediff("creationdate_885D", "employedfrom_700D")) \
       .withColumn("dias_para_primeira_parcela", datediff("firstnonzeroinstldate_307D", "creationdate_885D"))


In [25]:
# Criação de flags
train_applprev_1 = train_applprev_1 \
  .withColumn("sem_aprovacao_flag", when(col("approvaldate_319D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_ativacao_flag", when(col("dateactivated_425D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_pagamento_flag", when(col("dtlastpmt_581D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_pagamento_total_flag", when(col("dtlastpmtallstes_3545839D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_emprego_flag", when(col("employedfrom_700D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_parcela_flag", when(col("firstnonzeroinstldate_307D").isNull(), 1).otherwise(0))


In [26]:
colunas_float = [
    "credamount_590A",
    "currdebt_94A",
    "outstandingdebt_522A",
    "mainoccupationinc_437A",
    "byoccupationinc_3656910L",
    "annuity_853A",
    "revolvingaccount_394A"
]

for c in colunas_float:
    train_applprev_1 = train_applprev_1.withColumn(c, col(c).cast("double"))


In [27]:
# Flags de nulo/cruzamento
train_applprev_1 = train_applprev_1 \
    .withColumn("mainoccupationinc_null_flag", when(col("mainoccupationinc_437A").isNull(), 1).otherwise(0)) \
    .withColumn("byoccupationinc_null_flag", when(col("byoccupationinc_3656910L").isNull(), 1).otherwise(0)) \
    .withColumn("tem_revolving_flag", when(col("revolvingaccount_394A").isNotNull(), 1).otherwise(0))



In [28]:
from pyspark.sql.functions import coalesce

train_applprev_1 = train_applprev_1.withColumn("divida_total",
                   coalesce(col("currdebt_94A"), lit(0)) + coalesce(col("outstandingdebt_522A"), lit(0)))


In [29]:
# Criação de flag
train_applprev_1 = train_applprev_1 \
  .withColumn("limite_cartao_credito_flag", when(col("approvaldate_319D").isNull(), 1).otherwise(0))

In [30]:
from pyspark.sql.functions import when

categoricas_com_nulos = [
    "credtype_587L",
    "familystate_726L",
    "inittransactioncode_279L"
]

for c in categoricas_com_nulos:
    train_applprev_1 = train_applprev_1.withColumn(
        c,
        when(col(c).isNull(), "Desconhecido").otherwise(col(c))
    )


In [31]:
from pyspark.sql.types import StringType

categoricas_todas = [
    "cancelreason_3545846M", "credtype_587L", "district_544M",
    "education_1138M", "familystate_726L", "inittransactioncode_279L",
    "postype_4733339M", "profession_152M", "rejectreason_755M", "rejectreasonclient_4145042M"
]

for c in categoricas_todas:
    train_applprev_1 = train_applprev_1.withColumn(c, col(c).cast(StringType()))


In [32]:
from pyspark.sql.functions import when, col

# 🔹 1. Tratamento de flags booleanas (transformar em 0/1)
train_applprev_1 = train_applprev_1 \
    .withColumn("isbidproduct_390L", when(col("isbidproduct_390L") == True, 1)
                                      .when(col("isbidproduct_390L") == False, 0)
                                      .otherwise(0)) \
    .withColumn("isdebitcard_527L", when(col("isdebitcard_527L") == True, 1)
                                     .when(col("isdebitcard_527L") == False, 0)
                                     .otherwise(0))

# 🔹 2. Tratamento de colunas de contagem (nulos → 0, cast para int)
cols_contagem = [
    "childnum_21L",
    "pmtnum_8L",
    "tenor_203L",
    "credacc_transactions_402L"
]

for c in cols_contagem:
    train_applprev_1 = train_applprev_1.withColumn(
        c,
        (when(col(c).isNull(), 0).otherwise(col(c))).cast("int")
    )

# 🔹 3. Tratamento de categorias discretas com valor numérico (nulos → -1, cast para int)
train_applprev_1 = train_applprev_1 \
    .withColumn("credacc_status_367L", (when(col("credacc_status_367L").isNull(), -1)
                                        .otherwise(col("credacc_status_367L"))).cast("int")) \
    .withColumn("status_219L", (when(col("status_219L").isNull(), -1)
                                .otherwise(col("status_219L"))).cast("int"))


In [33]:
from pyspark.sql.functions import col, when, lit, first, avg, collect_set

from pyspark.sql.functions import when, col

# Define as colunas que indicam histórico de crédito anterior
colunas_credito = [
    "credacc_actualbalance_314A",
    "credacc_maxhisbal_375A",
    "credacc_minhisbal_90A",
    "downpmt_134A",
    "maxdpdtolerance_577P"
]

# Cria uma condição para verificar se TODAS estão nulas
condicao_sem_credito = None
for c in colunas_credito:
    condicao_sem_credito = col(c).isNull() if condicao_sem_credito is None else (condicao_sem_credito & col(c).isNull())

# Cria a flag binária
train_applprev_1 = train_applprev_1.withColumn(
    "sem_historico_credito_flag",
    when(condicao_sem_credito, 1).otherwise(0)
)

In [34]:
# Categóricas: preenchimento de nulo e cast para string
colunas_categoricas = [
    "district_544M",
    "education_1138M",
    "profession_152M",
    "postype_4733339M",
    "rejectreason_755M",
    "rejectreasonclient_4145042M"
]

for c in colunas_categoricas:
    train_applprev_1 = train_applprev_1.withColumn(c, when(col(c).isNull(), "Desconhecido").otherwise(col(c)).cast("string"))

In [35]:
train_applprev_1.coalesce(1).write.mode("overwrite").parquet(r"C:\Users\fred\meu_projeto_etl\data\interim\train_applprev_1.parquet")

## Tratamento da tabela train_applprev_2

In [38]:
train_applprev_2 = spark.read.parquet(r"C:\Users\fred\meu_projeto_etl\data\raw\train\train_applprev_2.parquet")

In [40]:
# Executar
tabela_resultado, resumo = avaliar_dataset(
    df=train_applprev_2,
    pk="case_id",
    colunas_data=[])

# Visualizar a tabela
import pandas as pd
from IPython.display import display
display(tabela_resultado)

# Exibir resumo
print(resumo)


📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 1182243 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,1221522,"[29, 964, 2453]",
1,cacccardblochreas_147M,string,0.78,109249,10,"[P19_60_110, P41_107_150, P33_145_161]",
2,conts_type_509L,string,17.01,2394056,10,"[EMPLOYMENT_PHONE, WHATSAPP, PRIMARY_MOBILE]",
3,credacc_cards_status_52L,string,97.57,13733404,7,"[UNCONFIRMED, INACTIVE, BLOCKED]",
4,num_group1,bigint,0.00,0,20,"[19, 0, 7]",
5,num_group2,bigint,0.00,0,12,"[0, 7, 6]",


{'total_registros': 14075487, 'colunas_com_nulos': 3, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 1182243}


### Análise da tabela train_applprev_2
#### Não foram tomadas ações sobre a tabela por baixa relevância de suas informações e impossibilidade de criação de chave primária

## Tratamento da tabela train_credit_bureau_a_1

In [53]:
train_credit_bureau_a_1 = spark.read.parquet(r"C:\Users\fred\meu_projeto_etl\data\raw\train\train_credit_bureau_a_1_*.parquet")

In [56]:
# Filtrar e selecionar
tabela_descritiva_filtrada = tabela_descritiva.filter(
    col('tabela') == 'train_credit_bureau_a_1_0'
).select('coluna', 'tipo_dado', 'Descricao')

# Mostrar para inspeção
tabela_descritiva_filtrada.show(45, truncate=False)

# Transformar em Pandas
tabela_descritiva_pd = tabela_descritiva_filtrada.toPandas()

# Salvar como Markdown
md_path = r"C:\Users\fred\Documents\Estudo de dados\projeto_credito\docs\tabela_descritiva_train_credit_bureau_a_1.md"

with open(md_path, "w", encoding="utf-8") as f:
    f.write(tabela_descritiva_pd.to_markdown(index=False))


+-------------------------------+---------+-------------------------------------------------------------------------------------------+
|coluna                         |tipo_dado|Descricao                                                                                  |
+-------------------------------+---------+-------------------------------------------------------------------------------------------+
|case_id                        |bigint   |NaN                                                                                        |
|annualeffectiverate_199L       |double   |Taxa de juros dos contratos fechados.                                                      |
|annualeffectiverate_63L        |double   |Taxa de juros para os contratos ativos.                                                    |
|classificationofcontr_13M      |string   |Classificação do contrato ativo.                                                           |
|classificationofcontr_400M     |string   |Class

In [58]:
# Executar
tabela_resultado, resumo = avaliar_dataset(
    df=train_credit_bureau_a_1,
    pk="case_id",
    colunas_data=["approvaldate_319D", "creationdate_885D", "dateactivated_425D", "dtlastpmt_581D","dtlastpmtallstes_3545839D", "employedfrom_700D", "firstnonzeroinstldate_307D"])

# Visualizar a tabela
import pandas as pd
from IPython.display import display
display(tabela_resultado)

# Exibir resumo
print(resumo)

📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 1386081 duplicatas com base na chave 'case_id'

📅 Amostras de colunas de data (sem nulos):
⚠️ Coluna approvaldate_319D não encontrada.
⚠️ Coluna creationdate_885D não encontrada.
⚠️ Coluna dateactivated_425D não encontrada.
⚠️ Coluna dtlastpmt_581D não encontrada.
⚠️ Coluna dtlastpmtallstes_3545839D não encontrada.
⚠️ Coluna employedfrom_700D não encontrada.
⚠️ Coluna firstnonzeroinstldate_307D não encontrada.


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,1386273,"[964, 1697, 1806]",
1,annualeffectiverate_199L,double,95.44,15213904,6598,"[9.13, 46.15, 13.4]",
2,annualeffectiverate_63L,double,98.05,15629294,5654,"[17.56, 41.89, 9.13]",
3,classificationofcontr_13M,string,0.00,0,11,"[be7b251d, ea6782cc, 1cf4e481]",
4,classificationofcontr_400M,string,0.00,0,389,"[51590aa9, acba4f13, ffee884a]",
...,...,...,...,...,...,...,...
74,totalamount_996A,double,91.19,14536246,283564,"[7747.1743, 300000.0, 197134.4]",
75,totaldebtoverduevalue_178A,double,91.81,14635534,51535,"[7565.216, 1406.9241, 2043.5801]",
76,totaldebtoverduevalue_718A,double,92.00,14664995,951,"[40637.5, 2.4, 65.200005]",
77,totaloutstanddebtvalue_39A,double,91.81,14635534,1081141,"[91101.96, 3130.3062, 13.2699995]",


{'total_registros': 15940537, 'colunas_com_nulos': 66, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 1386081}
